In [4]:
from bs4 import BeautifulSoup
import requests
from operator import itemgetter
import json
import datetime

url = 'https://www.bilibili.com/ranking'
html = requests.get(url).text
soup = BeautifulSoup(html,'lxml')
ul = soup.find_all(name='ul')
_all = ul[4].find_all(name = 'li')
index=[]
title=[]
view =[]
play =[]
author=[]
src=[]
for video in _all:
    index.append(video.find(class_='num').string)
    title.append(video.find(class_='title').string)
    view.append(video.find(class_='b-icon view').next_sibling.string)
    play.append(video.find(class_='b-icon play').next_sibling.string)
    author.append(video.find(class_='b-icon author').next_sibling.string)
    src.append(video.find(class_ = 'title').attrs['href'])

for a in range(len(view)):
    if(view[a][-1] =='万'):
        view[a] = int(float(view[a][:-1]) *10000)
    else:
        view[a] = int(view[a])
for b in range(len(play)):
    if(play[b][-1] == '万'):
        play[b] = int(float(play[b][:-1])*10000)
    else:
        play[b] = int(play[b])
index = list(map(int,index))
video = []
for i in range(len(_all)):
    temp = {'排名':index[i],'标题':title[i],'作者':author[i],'播放':play[i],'弹幕数':view[i],'来源':src[i]}
    video.append(temp)

In [21]:
import csv

with open('写入文件\\data.csv','w',encoding='utf-8') as f:
    fieldnames=['排名','标题','作者','播放','弹幕数','来源']
    writer = csv.DictWriter(f,fieldnames = fieldnames)
    for item in video:
        writer.writerow(item)

In [9]:
with open('写入文件\\data.csv','r',encoding='utf-8') as f:
    reader = csv.reader(f)

# 数据库存储 Mysql

In [16]:
import pymysql

db=pymysql.connect(host="localhost",user="root",password="1170",port=3306, db = 'spiders')
cursor = db.cursor()
cursor.execute('SELECT VERSION()')
data=cursor.fetchone()
sql = 'CREATE TABLE IF NOT EXISTS bilibili_rank ('\
      'id INT NOT NULL AUTO_INCREMENT,' \
      'rank_ VARCHAR(3),title VARCHAR(255),author VARCHAR(255), ' \
      'play INT, view INT, src VARCHAR(255),' \
      'PRIMARY KEY (id)) '
cursor.execute(sql)
db.close()

数据库版本 ('8.0.16',)


In [13]:
import pymysql

id = '20200312'
user = 'Bob'
age = 20
db=pymysql.connect(host="localhost",user="root",password="1170",port=3306, db = 'spiders')
cursor = db.cursor()
sql = 'INSERT INTO students(id,name,age) values(%s,%s,%s)'
try:
    cursor.execute(sql,(id,user,age))
    db.commit()
except:
    db.rollback()
db.close

<bound method Connection.close of <pymysql.connections.Connection object at 0x0000022E56476648>>

数据库操作标准写法：
  try :
      cursor.execute(sql)
      db.commit()
 except:
     db.rollback()
 db.close()

### 在数据库中插入字典(主键存在则更新，不存在则插入)

In [77]:
import json
import pymysql
#读取文件
with open('写入文件\\2020-03-11.json','r',encoding = 'utf-8') as f:
    str = f.read()
    reader = json.loads(str)
#建立数据库连接
db=pymysql.connect(host="localhost",user="root",password="1170",port=3306, db = 'spiders',charset="utf8")
cursor = db.cursor()

table = 'bilibili_rank'
keys = 'rank_,title,author,play,view,src'
k = ['rank_','title','author','play','view','src']
values = ','.join(['%s']*len(reader[0]))
sql = 'REPLACE INTO {table}({keys}) VALUES({values})'.format(table = table,keys = keys,values = values)
try:
    for i in range(len(reader)):
        if cursor.execute(sql,tuple(reader[i].values())):
            db.commit()
    print("Successful")
except:
    print('Failed')
    db.rollback()
db.close()

Successful


### 更新数据库

In [45]:
db=pymysql.connect(host="localhost",user="root",password="1170",port=3306, db = 'spiders',charset="utf8")
cursor = db.cursor()

sql = 'UPDATE bilibili_rank SET rank_ = %s WHERE id = %s'

try:
    cursor.execute(sql,(1,1))
    db.commit()
except:
    db.rollback()
db.close()

### 删除数据

In [60]:
db=pymysql.connect(host="localhost",user="root",password="1170",port=3306, db = 'spiders',charset="utf8")
cursor = db.cursor()

condition = 'id > 1'
sql = 'DELETE FROM {table} WHERE {condition}'.format(table = table,condition = condition)
try:
    cursor.execute(sql)
    db.commit()
except:
    db.rollback()

### 查询数据

In [80]:
db=pymysql.connect(host="localhost",user="root",password="1170",port=3306, db = 'spiders',charset="utf8")
cursor = db.cursor()

condition = 'rank_  = 1'
table = 'bilibili_rank'
sql = 'select * from {table} where {condition}'.format(table = table,condition = condition)

try:
    cursor.execute(sql)
    print('Count:',cursor.rowcount)
    for item in cursor.fetchall():
        print(item[3])
    db.commit()
except:
    print('faild')
    db.rollback()

Count: 2
凉风Kaze
老番茄


# MongoDB

In [22]:
import pymongo
import json

#读取文件
with open('写入文件\\2020-03-11.json','r',encoding = 'utf-8') as f:
    str = f.read()
    reader = json.loads(str)
    
client = pymongo.MongoClient(host='localhost',port = 27017)
db = client.spider
collection = db.bilibili_rank
result = collection.insert_many(reader)
print(result)

### 高级查询

In [48]:
result = collection.find({'$where': 'obj.弹幕数>=10000'}).count()
# print(result)
# for item in result:
#     print(item)
# print(result)

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


### 排序

In [72]:
results = collection.find({'排名':{'$lt':10}}).sort('弹幕数',pymongo.DESCENDING).limit(10)
for item in results:
    print(item['标题']+'\t'+ item['作者'] ,item['弹幕数'])

【C菌】这就是我投资了500欧元的游戏, 真的太值了! 《恶魔从不撒谎》实况 | 完结	渗透之C君 76000
【老番茄】史上最骚杀手(第八集)	老番茄 57000
【师生对线】这网课上得就尼玛离谱！（第三集）	鹤吱菌 47000
【新番导视】9.9分预定！我的青春回来了！	凉风Kaze 40000
【最全】解读肖战的背后资本，我再也不想做肖战的视频了！！！	是桃大 39000
B站的同学们，大家好，我是罗翔	罗翔说刑法 26000
几个超级实用的网站，用好了你也可以阅片无数	凉风Kaze 25000
我的老婆们突然叫爸爸（配音模仿）	hanser 24000
这TM才叫网课！	特效小哥studio 21000
“每日一遍，智商再见”	枫泽雨畔 18000


### 删除

In [77]:
result = collection.delete_many({'来源': {'$exists': False}})
print(result)

{'n': 100, 'ok': 1.0}


E:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  """Entry point for launching an IPython kernel.


# Redis

In [81]:
from redis import StrictRedis

redis = StrictRedis(host='localhost',port = 6379,db = 0)
redis.set('name','Bob')
print(redis.get('name'))

b'Bob'


In [96]:
print(redis.set('name','Hello'),redis.set('time','second'))
print(redis.exists('name'))
print(redis.type('name'))
print(redis.keys('n*'))
print(redis.randomkey())
# print(redis.rename('fakeName','name'))n
print(redis.dbsize())
print(redis.expire('name',30))
print(redis.ttl('name'))

True True
1
b'string'
[b'name']
b'time'
2
True
30


In [104]:
print(redis.getset('name','MI')) #取值并设置新值
print(redis.mget('name','time'))
print(redis.setnx('newname','James'))
print(redis.rpush('list','a','b','c'))

b'MI'
[b'MI', b'second']
False
6
